In [1]:
import torch
import numpy as np
import torchvision
from torchvision import datasets as datasets

In [2]:
!wget -c https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py

--2021-06-27 09:55:19--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2813 (2.7K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   2.75K  --.-KB/s    in 0s      

2021-06-27 09:55:19 (49.6 MB/s) - ‘helper.py’ saved [2813/2813]



In [3]:

transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize(227),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [4]:
train_data = datasets.CIFAR10('./images', train=True,
                              download=True, transform=transforms)
test_data = datasets.CIFAR10('./images', train=False,
                             download=True, transform=transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64,shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64,shuffle=True)


Extracting ./images/cifar-10-python.tar.gz to ./images
Files already downloaded and verified


In [5]:
import torch.nn as nn
import torch.nn.functional as F
class AlexNet(nn.Module):
  def __init__(self, n_classes):
    super(AlexNet, self).__init__()
    self.pool = nn.MaxPool2d(3, 2)
    self.conv1 = nn.Conv2d(3, 96, kernel_size=(11,11), stride=4)
    self.conv2 = nn.Conv2d(96, 256, kernel_size=(5,5),padding=2)
    self.conv3 = nn.Conv2d(256, 384, kernel_size=(3,3),padding=1)
    self.conv4 = nn.Conv2d(384, 384, kernel_size=(3,3),padding=1)
    self.conv5 = nn.Conv2d(384, 256, kernel_size=(3,3),padding=1)
    self.fc1 = nn.Linear(256*6*6, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, n_classes)
    self.dropout = nn.Dropout(p = 0.5)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = self.pool(F.relu(self.conv5(x)))
    x = x.view(x.size(0),256*6*6)
    x = self.dropout(self.fc1(x))
    x = self.dropout(self.fc2(x))
    x = self.fc3(x)
    return x

In [6]:
model = AlexNet(n_classes=10)
model.to('cuda')

AlexNet(
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=9216, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
criterion = nn.CrossEntropyLoss() 
num_epochs = 20
for epoch in range(num_epochs):
    training_loss = 0
    for data in trainloader:
      images, labels = data
      images = images.to('cuda')
      labels = labels.to('cuda')
      output = model(images)
      optimizer.zero_grad()
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()
      training_loss += loss.item()
      
    print(f'Training loss : {training_loss/len(trainloader)}')



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Training loss : 2.2720954030980844
Training loss : 1.8110992638656245
Training loss : 1.529491868470331
Training loss : 1.3848453382854267
Training loss : 1.2498738755807852
Training loss : 1.1274675915155874
Training loss : 1.0088806148532712
Training loss : 0.9138890361160878
Training loss : 0.8284885807872733
Training loss : 0.7542868870146134
Training loss : 0.6929049839449051
Training loss : 0.6419012867428763
Training loss : 0.5869216276404193
Training loss : 0.5447225392703205
Training loss : 0.4966321719805603
Training loss : 0.45802073470314447
Training loss : 0.4182134490755513
Training loss : 0.3770304339773515
Training loss : 0.3438473294972611
Training loss : 0.30539364937473745


In [8]:
#Accuracy on Test set
running_accuracy = 0
total_accuracy = 0
for data in testloader:
  images, labels = data
  images = images.to('cuda')
  labels = labels.to('cuda')
  output = model(images)
  predicted_labels = torch.argmax(output, 1)
  accuracy = (predicted_labels == labels).sum()/len(labels)
  total_accuracy += accuracy/len(testloader)
print(f'{total_accuracy*100}%') 

80.1054916381836%
